In [1]:
import pandas as pd
import yfinance as yf
import time
import numpy as np

In [9]:
data = pd.read_csv("Results.csv", index_col= 0)
data = data.drop(columns=['Unnamed: 0'])
data.to_csv("Results.csv")

In [7]:
DF['Date Time'] = pd.to_datetime(DF['Date Time'])

In [11]:
data_5 = data
data_15 = data

In [12]:
data_5

,Date Time,Stock,Curr_Price,Buy,Sell,Hold,Day1_Pred,closest_day1_price,actual_day1_close,day1_min_error,...,Query,Advice,profit_1,profit_3,profit_7,Category,missed_opportunity_day1,missed_opportunity_day3,missed_opportunity_day7,Date
0,2025-07-03 00:00,NVDA,159.271103,0,1,0,162.50,159.160004,158.240005,2.098515,...,1. I bought Nvidia (NVDA) at $960 but it’s now...,"Continue riding the AI wave, but consider trim...",True,False,False,Technology,0,0,0,2025-07-03
1,2025-07-03 00:00,AMZN,223.289993,1,0,0,230.00,224.220001,223.479996,2.577825,...,2. With Amazon (AMZN) up after its Prime Day p...,"Buy now, as the stock is likely to continue it...",True,False,True,Consumer,0,1,0,2025-07-03
2,2025-07-03 00:00,TSLA,315.184998,0,0,1,310.23,295.929993,293.920013,4.832226,...,3. Tesla (TSLA) just announced another price c...,Hold,False,False,False,EV Auto,0,0,0,2025-07-03
3,2025-07-03 00:00,AMD,137.949997,1,0,0,145.00,137.175003,134.830002,5.704390,...,4. After the recent pullback in Advanced Micro...,"Buy, as the recent pullback in AMD presents an...",False,True,True,Technology,0,0,0,2025-07-03
4,2025-07-03 00:00,META,718.599976,0,0,1,732.00,726.505005,718.640015,0.756360,...,5. Is Meta Platforms (META) still a good buy g...,"{'stock': 'META', 'reason': 'Meta Platforms, I...",True,True,False,Technology,0,0,1,2025-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,2025-07-03 00:00,ROST,131.649994,0,0,1,95.21,129.899994,130.770004,26.705154,...,95. What’s the ideal weight for a 4-stock mix ...,"{'ideal_weight': {'ASML': 30, 'KLAC': 25, 'INT...",False,False,False,Consumer,0,0,0,2025-07-03
186,2025-07-03 00:00,EBAY,76.389999,0,1,0,74.50,75.288696,75.889999,1.047563,...,"99. I’ve held eBay for a while, but I’m consid...",{'coming_day': 'Hold your position in EBAY for...,True,True,True,Consumer,0,0,0,2025-07-03
187,2025-07-03 00:00,ADP,308.959991,0,0,0,313.00,310.725006,308.489990,0.732157,...,"100. I'm looking at ADP, NTES, and KLAC for lo...","Based on the analysis, I recommend investing i...",True,True,True,Technology,0,0,0,2025-07-03
188,2025-07-03 00:00,NTES,132.860001,1,0,0,137.00,134.154999,133.039993,2.120682,...,"100. I'm looking at ADP, NTES, and KLAC for lo...","Based on the analysis, I recommend investing i...",True,False,False,Technology,0,0,0,2025-07-03


In [15]:
data_5 = data_5.drop(columns=['closest_day1_price','closest_day3_price','closest_day7_price', 'actual_day1_close', 'actual_day3_close', 'actual_day7_close'])

In [19]:
t_date_1 = pd.to_datetime("07-07-2025", dayfirst=True, utc=True)
t_date_end_1 = pd.to_datetime("08-07-2025", dayfirst=True, utc=True)

next_date_3 = pd.to_datetime("09-07-2025", dayfirst=True, utc=True)
next_date_end_3 = pd.to_datetime("10-07-2025", dayfirst=True, utc=True)

next_date_7 = pd.to_datetime("11-07-2025", dayfirst=True, utc=True)
next_date_end_7 = pd.to_datetime("12-07-2025", dayfirst=True, utc=True)

for i, row in data_5.iterrows():
    stock = row['Stock']
    target_price_day7 = row['Day7_Pred']
    target_price_day3 = row['Day3_Pred']
    target_price_day1 = row['Day1_Pred']
    
    try:
        # Download 1-minute interval data for today (or modify date range as needed)
        market_today = yf.download(stock, period="1mo", interval="5m", timeout=45)
        market_today.columns = market_today.columns.droplevel(1)
        market_today = market_today.reset_index()[['Datetime', 'Close', 'High', 'Low', 'Open', 'Volume']]

        
        #From Day 1 -->next day
        market_today_day1 = market_today[
            (market_today['Datetime'] >= t_date_1) &
            (market_today['Datetime'] < t_date_end_1)
        ]
        # Find closest price to Day1
        market_today_day1['diff'] = (market_today_day1['Close'] - target_price_day1).abs()
        closest_price_day1 = market_today_day1.loc[market_today_day1['diff'].idxmin(), 'Close']
        # Get the last close price (actual close of the day)
        actual_day1_close = market_today_day1['Close'].iloc[-1]

        #From 3 Days
        market_today_day3 = market_today[
            (market_today['Datetime'] >= next_date_3) &
            (market_today['Datetime'] < next_date_end_3)
        ]
        # Find closest price to Day3
        market_today_day3['diff'] = (market_today_day3['Close'] - target_price_day3).abs()
        closest_price_day3 = market_today_day3.loc[market_today_day3['diff'].idxmin(), 'Close']

        # Get the last close price (actual close of the day)
        actual_day3_close = market_today_day3['Close'].iloc[-1]


        #From 7 Days
        market_today_day7 = market_today[
            (market_today['Datetime'] >= next_date_7) &
            (market_today['Datetime'] < next_date_end_7)
        ]
        # Find closest price to Day7
        market_today_day7['diff'] = (market_today_day7['Close'] - target_price_day7).abs()
        closest_price_day7 = market_today_day7.loc[market_today_day7['diff'].idxmin(), 'Close']

        # Get the last close price (actual close of the day)
        actual_day7_close = market_today_day7['Close'].iloc[-1]

    except Exception as e:
        print(f"Error for {stock}: {e}")
        closest_price_day3 = np.nan
        closest_price_day1 = np.nan
        actual_day1_close = np.nan
        closest_price_day7 = np.nan
        actual_day7_close = np.nan
        actual_day3_close = np.nan

    # Assign to new columns in data
    data_5.loc[i, 'closest_day3_price'] = closest_price_day3
    data_5.loc[i, 'actual_day3_close'] = actual_day3_close
    data_5.loc[i, 'closest_day1_price'] = closest_price_day1
    data_5.loc[i, 'actual_day1_close'] = actual_day1_close
    data_5.loc[i, 'closest_day7_price'] = closest_price_day7
    data_5.loc[i, 'actual_day7_close'] = actual_day7_close


C:\Users\Azeem\AppData\Local\Temp\ipykernel_14872\4060418710.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_today = yf.download(stock, period="1mo", interval="5m")
[*********************100%***********************]  1 of 1 completed
C:\Users\Azeem\AppData\Local\Temp\ipykernel_14872\4060418710.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_today_day1['diff'] = (market_today_day1['Close'] - target_price_day1).abs()
C:\Users\Azeem\AppData\Local\Temp\ipykernel_14872\4060418710.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

Error for QCOM: "['Datetime'] not in index"


[*********************100%***********************]  1 of 1 completed
C:\Users\Azeem\AppData\Local\Temp\ipykernel_14872\4060418710.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_today_day1['diff'] = (market_today_day1['Close'] - target_price_day1).abs()
C:\Users\Azeem\AppData\Local\Temp\ipykernel_14872\4060418710.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_today_day3['diff'] = (market_today_day3['Close'] - target_price_day3).abs()
C:\Users\Azeem\AppData\Local\Temp\ipykernel_14872

Error for MSFT: "['Datetime'] not in index"


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['NVDA']: Timeout('Failed to perform, curl: (28) Operation timed out after 10012 milliseconds with 9104 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
C:\Users\Azeem\AppData\Local\Temp\ipykernel_14872\4060418710.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_today = yf.download(stock, period="1mo", interval="5m")


...

[*********************100%***********************]  2 of 2 completed

2 Failed downloads:
['MICROCHIP', 'TECHNOLOGY']: YFPricesMissingError('possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")')
C:\Users\Azeem\AppData\Local\Temp\ipykernel_14872\4060418710.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_today = yf.download(stock, period="1mo", interval="5m")


Error for Microchip Technology: "['Datetime'] not in index"


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['SEAGEN']: YFPricesMissingError('possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")')
C:\Users\Azeem\AppData\Local\Temp\ipykernel_14872\4060418710.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_today = yf.download(stock, period="1mo", interval="5m")


Error for Seagen: "['Datetime'] not in index"


[*********************100%***********************]  1 of 1 completed
C:\Users\Azeem\AppData\Local\Temp\ipykernel_14872\4060418710.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_today_day1['diff'] = (market_today_day1['Close'] - target_price_day1).abs()
C:\Users\Azeem\AppData\Local\Temp\ipykernel_14872\4060418710.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_today_day3['diff'] = (market_today_day3['Close'] - target_price_day3).abs()
C:\Users\Azeem\AppData\Local\Temp\ipykernel_14872

In [ ]:
t_date_1 = pd.to_datetime("07-07-2025", dayfirst=True, utc=True)
t_date_end_1 = pd.to_datetime("08-07-2025", dayfirst=True, utc=True)

next_date_3 = pd.to_datetime("09-07-2025", dayfirst=True, utc=True)
next_date_end_3 = pd.to_datetime("10-07-2025", dayfirst=True, utc=True)

next_date_7 = pd.to_datetime("11-07-2025", dayfirst=True, utc=True)
next_date_end_7 = pd.to_datetime("12-07-2025", dayfirst=True, utc=True)

for i, row in data_15.iterrows():
    stock = row['Stock']
    target_price_day7 = row['Day7_Pred']
    target_price_day3 = row['Day3_Pred']
    target_price_day1 = row['Day1_Pred']
    
    try:
        # Download 1-minute interval data for today (or modify date range as needed)
        market_today = yf.download(stock, period="1mo", interval="15m")
        market_today.columns = market_today.columns.droplevel(1)
        market_today = market_today.reset_index()[['Datetime', 'Close', 'High', 'Low', 'Open', 'Volume']]

        
        #From Day 1 -->next day
        market_today_day1 = market_today[
            (market_today['Datetime'] >= t_date_1) &
            (market_today['Datetime'] < t_date_end_1)
        ]
        # Find closest price to Day1
        market_today_day1['diff'] = (market_today_day1['Close'] - target_price_day1).abs()
        closest_price_day1 = market_today_day1.loc[market_today_day1['diff'].idxmin(), 'Close']
        # Get the last close price (actual close of the day)
        actual_day1_close = market_today_day1['Close'].iloc[-1]

        #From 3 Days
        market_today_day3 = market_today[
            (market_today['Datetime'] >= next_date_3) &
            (market_today['Datetime'] < next_date_end_3)
        ]
        # Find closest price to Day3
        market_today_day3['diff'] = (market_today_day3['Close'] - target_price_day3).abs()
        closest_price_day3 = market_today_day3.loc[market_today_day3['diff'].idxmin(), 'Close']

        # Get the last close price (actual close of the day)
        actual_day3_close = market_today_day3['Close'].iloc[-1]


        #From 7 Days
        market_today_day7 = market_today[
            (market_today['Datetime'] >= next_date_7) &
            (market_today['Datetime'] < next_date_end_7)
        ]
        # Find closest price to Day7
        market_today_day7['diff'] = (market_today_day7['Close'] - target_price_day7).abs()
        closest_price_day7 = market_today_day7.loc[market_today_day7['diff'].idxmin(), 'Close']

        # Get the last close price (actual close of the day)
        actual_day7_close = market_today_day7['Close'].iloc[-1]

    except Exception as e:
        print(f"Error for {stock}: {e}")
        closest_price_day3 = np.nan
        closest_price_day1 = np.nan
        actual_day1_close = np.nan
        closest_price_day7 = np.nan
        actual_day7_close = np.nan
        actual_day3_close = np.nan

    # Assign to new columns in data
    data_15.loc[i, 'closest_day3_price'] = closest_price_day3
    data_15.loc[i, 'actual_day3_close'] = actual_day3_close
    data_15.loc[i, 'closest_day1_price'] = closest_price_day1
    data_15.loc[i, 'actual_day1_close'] = actual_day1_close
    data_15.loc[i, 'closest_day7_price'] = closest_price_day7
    data_15.loc[i, 'actual_day7_close'] = actual_day7_close


In [24]:
data.head(5)

,Date Time,Stock,Curr_Price,Buy,Sell,Hold,Day1_Pred,closest_day1_price,actual_day1_close,day1_min_error,...,Query,Advice,profit_1,profit_3,profit_7,Category,missed_opportunity_day1,missed_opportunity_day3,missed_opportunity_day7,Date
0,2025-07-03 00:00,NVDA,159.271103,0,1,0,162.50,158.770004,158.240005,2.098515,...,1. I bought Nvidia (NVDA) at $960 but it’s now...,"Continue riding the AI wave, but consider trim...",True,False,False,Technology,0,0,0,2025-07-03
1,2025-07-03 00:00,AMZN,223.289993,1,0,0,230.00,224.169998,223.479996,2.577825,...,2. With Amazon (AMZN) up after its Prime Day p...,"Buy now, as the stock is likely to continue it...",True,False,True,Consumer,0,1,0,2025-07-03
2,2025-07-03 00:00,TSLA,315.184998,0,0,1,310.23,295.079803,293.920013,4.832226,...,3. Tesla (TSLA) just announced another price c...,Hold,False,False,False,EV Auto,0,0,0,2025-07-03
3,2025-07-03 00:00,AMD,137.949997,1,0,0,145.00,136.441299,134.830002,5.704390,...,4. After the recent pullback in Advanced Micro...,"Buy, as the recent pullback in AMD presents an...",False,True,True,Technology,0,0,0,2025-07-03
4,2025-07-03 00:00,META,718.599976,0,0,1,732.00,NaN,NaN,0.756360,...,5. Is Meta Platforms (META) still a good buy g...,"{'stock': 'META', 'reason': 'Meta Platforms, I...",True,True,False,Technology,0,0,1,2025-07-03


In [9]:
DF['day7_min_error'] = abs(DF['Day7_Pred'] - DF['closest_day7_price']) * 100/ DF['closest_day7_price']
DF['day7_act_error'] = abs(DF['Day7_Pred'] - DF['actual_day7_close']) * 100/ DF['actual_day7_close']
DF['made_profit_act_day_7'] = (DF['Day7_Pred'] - DF['actual_day7_close'] > 0).astype(int)
DF['made_profit_could_day_7'] = (DF['Day7_Pred'] - DF['closest_day7_price'] > 0).astype(int)

In [7]:
prev = DF[['Stock','made_profit_act_day_1', 'made_profit_could_day_1']]
prev

,Stock,made_profit_act_day_1,made_profit_could_day_1
0,NVDA,1,1
1,AMZN,1,1
2,TSLA,1,1
3,AMD,1,1
4,META,1,1
...,...,...,...
185,ROST,0,0
186,EBAY,0,0
187,ADP,1,1
188,NTES,1,1


In [9]:
new = DF[['Stock','made_profit_act_day_1', 'made_profit_could_day_1']]
new

,Stock,made_profit_act_day_1,made_profit_could_day_1
0,NVDA,1,1
1,AMZN,1,1
2,TSLA,0,0
3,AMD,0,0
4,META,1,1
...,...,...,...
185,ROST,0,0
186,EBAY,1,1
187,ADP,0,0
188,NTES,1,1


In [11]:
new.describe()

,made_profit_act_day_1,made_profit_could_day_1
count,190.000000,190.000000
mean,0.389474,0.521053
std,0.488919,0.500876
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,1.000000
75%,1.000000,1.000000
max,1.000000,1.000000


In [12]:
DF['made_profit_act_day_1'] = (
    ((DF['Buy'] == 1) & (DF['actual_day1_close'] > DF['Curr_Price'])) |
    ((DF['Sell'] == 1) & (DF['actual_day1_close'] < DF['Curr_Price'])) |
    ((DF['Hold'] == 1) & (DF['actual_day1_close'] >= DF['Curr_Price']))
).astype(int)

DF['made_profit_could_day_1'] = (
    ((DF['Buy'] == 1) & (DF['closest_day1_price'] > DF['Curr_Price'])) |
    ((DF['Sell'] == 1) & (DF['closest_day1_price'] < DF['Curr_Price'])) |
    ((DF['Hold'] == 1) & (DF['closest_day1_price'] >= DF['Curr_Price']))
).astype(int)

DF['made_profit_act_day_3'] = (
    ((DF['Buy'] == 1) & (DF['actual_day3_close'] > DF['Curr_Price'])) |
    ((DF['Sell'] == 1) & (DF['actual_day3_close'] < DF['Curr_Price'])) |
    ((DF['Hold'] == 1) & (DF['actual_day3_close'] >= DF['Curr_Price']))
).astype(int)

DF['made_profit_could_day_3'] = (
    ((DF['Buy'] == 1) & (DF['closest_day3_price'] > DF['Curr_Price'])) |
    ((DF['Sell'] == 1) & (DF['closest_day3_price'] < DF['Curr_Price'])) |
    ((DF['Hold'] == 1) & (DF['closest_day3_price'] >= DF['Curr_Price']))
).astype(int)

DF['made_profit_act_day_7'] = (
    ((DF['Buy'] == 1) & (DF['actual_day7_close'] > DF['Curr_Price'])) |
    ((DF['Sell'] == 1) & (DF['actual_day7_close'] < DF['Curr_Price'])) |
    ((DF['Hold'] == 1) & (DF['actual_day7_close'] >= DF['Curr_Price']))
).astype(int)

DF['made_profit_could_day_7'] = (
    ((DF['Buy'] == 1) & (DF['closest_day7_price'] > DF['Curr_Price'])) |
    ((DF['Sell'] == 1) & (DF['closest_day7_price'] < DF['Curr_Price'])) |
    ((DF['Hold'] == 1) & (DF['closest_day7_price'] >= DF['Curr_Price']))
).astype(int)


In [16]:
# Mean of profit made (actual and could-have-been)
mean_profits = {
    'mean_profit_act_day_1': DF['made_profit_act_day_1'].mean(),
    'mean_profit_could_day_1': DF['made_profit_could_day_1'].mean(),
    'mean_profit_act_day_3': DF['made_profit_act_day_3'].mean(),
    'mean_profit_could_day_3': DF['made_profit_could_day_3'].mean(),
    'mean_profit_act_day_7': DF['made_profit_act_day_7'].mean(),
    'mean_profit_could_day_7': DF['made_profit_could_day_7'].mean()
}

# Display the result
for k, v in mean_profits.items():
    print(f"{k}: {v:.3f}")


mean_profit_act_day_1: 0.389
mean_profit_could_day_1: 0.521
mean_profit_act_day_3: 0.389
mean_profit_could_day_3: 0.453
mean_profit_act_day_7: 0.342
mean_profit_could_day_7: 0.379


In [15]:
print("People Who Depended Entirely For Day1: ", DF['made_profit_act_day_1'].mean() * 100)
print("People Who Monitered For Day1: ", DF['made_profit_could_day_1'].mean() * 100)
print("People Who Depended Entirely For Day3: ", DF['made_profit_act_day_3'].mean() * 100)
print("People Who Monitered For Day3: ", DF['made_profit_could_day_3'].mean() * 100)
print("People Who Depended Entirely For Day7: ", DF['made_profit_act_day_7'].mean() * 100)
print("People Who Monitered For Day7: ", DF['made_profit_could_day_7'].mean() * 100)

People Who Depended Entirely For Day1:  38.94736842105263
People Who Monitered For Day1:  52.10526315789473
People Who Depended Entirely For Day3:  38.94736842105263
People Who Monitered For Day3:  45.26315789473684
People Who Depended Entirely For Day7:  34.21052631578947
People Who Monitered For Day7:  37.89473684210527


In [56]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import os, re, json
import pandas as pd

load_dotenv()
MODEL = "llama-3.1-8b-instant"

llm = ChatGroq(
    temperature=0,
    model_name=MODEL,
    api_key=os.environ.get("GROQ_API_KEY"),
)


In [83]:

rem =list(set(advices[155:]))
len(rem)

12

In [84]:



for i, adv in enumerate(rem):

    prompt = f"""
        You are an expert financial assistant.

        The user has provided a tuple containing a **stock market query** and an **AI-generated investment advice**. Your task is to:

        1. Identify the stock symbol(s) mentioned.
        2. Determine the recommended action: **Buy**, **Sell**, or **Hold**.
        - Use "Sell" if advice suggests reducing/trimming the position.
        - Use "Buy" if the advice implies increasing/investing in the stock.
        - Use "Hold" if advice suggests staying invested without any changes.

        ---

        Example Input:
        Query: "I bought Nvidia (NVDA) at $960 but it’s now near $1,200. Should I trim my position or continue riding the AI wave?"
        Advice: "Continue riding the AI wave, but consider trimming your position by 10-20% to lock in some profits."

        Expected Output:
        {{
        "NVDA": "Sell"
        }}

        ---

        Now process the following input and return **only a valid JSON object**:

        Query: {adv[0]}
        Advice: {adv[1]}
    """


    resoponce = llm.invoke([HumanMessage(prompt)])
    output = resoponce.content
    match = re.search(r"\{[^{}]+\}", output)
    if match:
        try:
            result_dict = json.loads(match.group())
            act.update({adv[0] : result_dict})
            print("Added: ", i)
        except json.JSONDecodeError as e:
            print(f"JSON parsing failed: {e}")
    else:
        print("No JSON object found in the output.")

Added:  0
Added:  1
Added:  2
JSON parsing failed: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Added:  4
Added:  5
Added:  6
Added:  7
Added:  8
Added:  9
Added:  10
Added:  11


In [106]:
DF = DF.drop(columns=['Buy', 'Sell', 'Hold'], axis = 1)
for col in ['Buy', 'Sell', 'Hold']:
    if col not in DF.columns:
        DF[col] = 0
DF

,Query,Stock,Day1,Day3,Day7,Date Time,Advice,closest_day3_price,actual_day3_close,closest_day1_price,...,day1_act_error,made_profit_act_day_1,made_profit_could_day_1,day3_min_error,day3_act_error,made_profit_act_day_3,made_profit_could_day_3,Buy,Sell,Hold
0,1. I bought Nvidia (NVDA) at $960 but it’s now...,NVDA,162.50,165.80,170.20,05-07-2024 14:30,"Continue riding the AI wave, but consider trim...",164.285004,162.830002,159.160004,...,2.692110,1,1,0.922176,1.823987,1,1,0,0,0
1,2. With Amazon (AMZN) up after its Prime Day p...,AMZN,230.00,235.00,240.00,05-07-2024 14:30,"Buy now, as the stock is likely to continue it...",224.199997,222.470001,224.220001,...,2.917489,1,1,4.817129,5.632219,1,1,0,0,0
2,3. Tesla (TSLA) just announced another price c...,TSLA,310.23,308.56,306.89,05-07-2024 14:30,Hold,299.359985,295.820007,295.929993,...,5.549124,1,1,3.073228,4.306670,1,1,0,0,0
3,4. After the recent pullback in Advanced Micro...,AMD,145.00,150.00,155.00,05-07-2024 14:30,"Buy, as the recent pullback in AMD presents an...",140.479095,138.449997,137.175003,...,7.542830,1,1,6.777453,8.342364,1,1,0,0,0
4,5. Is Meta Platforms (META) still a good buy g...,META,732.00,745.00,760.00,05-07-2024 14:30,"{'stock': 'META', 'reason': 'Meta Platforms, I...",737.109985,732.799988,726.505005,...,1.859065,1,1,1.070399,1.664849,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,95. What’s the ideal weight for a 4-stock mix ...,ROST,95.21,96.58,98.01,06-07-2024 14:30,"{'ideal_weight': {'ASML': 30, 'KLAC': 25, 'INT...",130.690002,131.210007,129.899994,...,27.192784,0,0,26.099933,26.392809,0,0,0,0,0
109,"99. I’ve held eBay for a while, but I’m consid...",EBAY,74.50,73.20,72.50,06-07-2024 14:30,{'coming_day': 'Hold your position in EBAY for...,75.170799,76.279999,75.288696,...,1.831598,0,0,2.621762,4.037754,0,0,0,0,0
110,"100. I'm looking at ADP, NTES, and KLAC for lo...",ADP,313.00,316.00,320.00,06-07-2024 14:30,"Based on the analysis, I recommend investing i...",308.612488,308.410004,310.725006,...,1.461963,1,1,2.393783,2.461008,1,1,0,0,0
111,"100. I'm looking at ADP, NTES, and KLAC for lo...",NTES,137.00,140.00,144.00,06-07-2024 14:30,"Based on the analysis, I recommend investing i...",131.050003,130.199997,134.154999,...,2.976554,1,1,6.829452,7.526884,1,1,0,0,0


In [107]:
for query_text, stock_actions in act.items():
    for stock, action in stock_actions.items():
        if action in ['Buy', 'Sell', 'Hold']:
            DF.loc[(DF['Stock'] == stock) & (DF['Query'] == query_text), action] = 1

In [5]:
DF = pd.read_csv('Results.csv')
stocks = list(set(DF['Stock']))

for col in ['Curr_Price']:
    if col not in DF.columns:
        DF[col] = 0




In [11]:
date_ = pd.to_datetime("03-07-2025", dayfirst=True, utc=True)
date_end =pd.to_datetime("04-07-2025", dayfirst=True, utc=True)

for stock in stocks:
    try:
        market_today = yf.download(stock, period="5d", interval="1m")
        market_today.columns = market_today.columns.droplevel(1)
        market_today = market_today.reset_index()[['Datetime', 'Close', 'High', 'Low', 'Open', 'Volume']]
        slice = market_today[(market_today['Datetime'] >= date_) & (market_today['Datetime']< date_end)]

        DF.loc[DF['Stock'] == stock, 'Curr_Price'] = slice['Close'].iloc[-1]

    except Exception as e:
        print("Error:", stock, "\n", e)
        continue

C:\Users\Azeem\AppData\Local\Temp\ipykernel_21596\87630360.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_today = yf.download(stock, period="5d", interval="1m")
[*********************100%***********************]  1 of 1 completed
C:\Users\Azeem\AppData\Local\Temp\ipykernel_21596\87630360.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '78.9749984741211' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  DF.loc[DF['Stock'] == stock, 'Curr_Price'] = slice['Close'].iloc[-1]
C:\Users\Azeem\AppData\Local\Temp\ipykernel_21596\87630360.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_today = yf.download(stock, period="5d", interval="1m")
[*********************100%***********************]  1 of 1 completed
C:\Users\Azeem\AppData\Local\Temp\ipykernel_21596\87630360.py:6: FutureWarnin

Error: Microchip Technology 
 "['Datetime'] not in index"


[*********************100%***********************]  1 of 1 completed
C:\Users\Azeem\AppData\Local\Temp\ipykernel_21596\87630360.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_today = yf.download(stock, period="5d", interval="1m")
[*********************100%***********************]  1 of 1 completed
C:\Users\Azeem\AppData\Local\Temp\ipykernel_21596\87630360.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_today = yf.download(stock, period="5d", interval="1m")
[*********************100%***********************]  1 of 1 completed
C:\Users\Azeem\AppData\Local\Temp\ipykernel_21596\87630360.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_today = yf.download(stock, period="5d", interval="1m")
[*********************100%***********************]  1 of 1 completed
C:\Users\Azeem\AppData\Local\Temp\ipykernel_21596\87630360.py:6: FutureWarning: YF.download() has chang

Error: Intuit 
 "['Datetime'] not in index"


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['SEAGEN']: YFPricesMissingError('possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")')
C:\Users\Azeem\AppData\Local\Temp\ipykernel_21596\87630360.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_today = yf.download(stock, period="5d", interval="1m")


...

[*********************100%***********************]  1 of 1 completed
C:\Users\Azeem\AppData\Local\Temp\ipykernel_21596\87630360.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_today = yf.download(stock, period="5d", interval="1m")
[*********************100%***********************]  1 of 1 completed
C:\Users\Azeem\AppData\Local\Temp\ipykernel_21596\87630360.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_today = yf.download(stock, period="5d", interval="1m")
[*********************100%***********************]  1 of 1 completed
C:\Users\Azeem\AppData\Local\Temp\ipykernel_21596\87630360.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_today = yf.download(stock, period="5d", interval="1m")
[*********************100%***********************]  1 of 1 completed
C:\Users\Azeem\AppData\Local\Temp\ipykernel_21596\87630360.py:6: FutureWarning: YF.download() has chang

Error: CoStar Group 
 "['Datetime'] not in index"


[*********************100%***********************]  1 of 1 completed
C:\Users\Azeem\AppData\Local\Temp\ipykernel_21596\87630360.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_today = yf.download(stock, period="5d", interval="1m")
[*********************100%***********************]  2 of 2 completed

2 Failed downloads:
['LAM']: YFPricesMissingError('possibly delisted; no price data found  (period=5d)')
['RESEARCH']: YFPricesMissingError('possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")')
C:\Users\Azeem\AppData\Local\Temp\ipykernel_21596\87630360.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_today = yf.download(stock, period="5d", interval="1m")


Error: Lam Research 
 "['Datetime'] not in index"


[*********************100%***********************]  1 of 1 completed
C:\Users\Azeem\AppData\Local\Temp\ipykernel_21596\87630360.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_today = yf.download(stock, period="5d", interval="1m")
[*********************100%***********************]  1 of 1 completed
C:\Users\Azeem\AppData\Local\Temp\ipykernel_21596\87630360.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_today = yf.download(stock, period="5d", interval="1m")
[*********************100%***********************]  1 of 1 completed
C:\Users\Azeem\AppData\Local\Temp\ipykernel_21596\87630360.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_today = yf.download(stock, period="5d", interval="1m")
[*********************100%***********************]  1 of 1 completed
C:\Users\Azeem\AppData\Local\Temp\ipykernel_21596\87630360.py:6: FutureWarning: YF.download() has chang

In [11]:
DF.to_csv("Results.csv")

In [12]:
DF.columns

Index(['Unnamed: 0', 'Date Time', 'Stock', 'Curr_Price', 'Buy', 'Sell', 'Hold',
       'Day1_Pred', 'closest_day1_price', 'actual_day1_close',
       'day1_min_error', 'day1_act_error', 'made_profit_could_day_1',
       'made_profit_act_day_1', 'Day3_Pred', 'closest_day3_price',
       'actual_day3_close', 'day3_min_error', 'day3_act_error',
       'made_profit_act_day_3', 'made_profit_could_day_3', 'Day7_Pred',
       'Query', 'Advice', 'closest_day7_price', 'actual_day7_close',
       'day7_min_error', 'day7_act_error', 'made_profit_act_day_7',
       'made_profit_could_day_7'],
      dtype='object')

In [13]:
DF = DF[['Date Time','Stock', 'Curr_Price','Buy', 'Sell', 'Hold', 
        'Day1_Pred','closest_day1_price', 'actual_day1_close', 'day1_min_error','day1_act_error', 'made_profit_could_day_1','made_profit_act_day_1',
        'Day3_Pred','closest_day3_price', 'actual_day3_close', 'day3_min_error', 'day3_act_error', 'made_profit_act_day_3', 'made_profit_could_day_3'
        ,'Day7_Pred','closest_day7_price', 'actual_day7_close', 'day7_min_error','day7_act_error', 'made_profit_could_day_7','made_profit_act_day_7','Query', 'Advice']]



In [14]:
DF.to_csv("Results.csv")

In [15]:
DF

,Date Time,Stock,Curr_Price,Buy,Sell,Hold,Day1_Pred,closest_day1_price,actual_day1_close,day1_min_error,...,made_profit_could_day_3,Day7_Pred,closest_day7_price,actual_day7_close,day7_min_error,day7_act_error,made_profit_could_day_7,made_profit_act_day_7,Query,Advice
0,05-07-2024 14:30,NVDA,159.271103,0,1,0,162.50,159.160004,158.240005,2.098515,...,1,170.20,167.500000,167.359894,1.611940,1.697005,1,1,1. I bought Nvidia (NVDA) at $960 but it’s now...,"Continue riding the AI wave, but consider trim..."
1,05-07-2024 14:30,AMZN,223.289993,1,0,0,230.00,224.220001,223.479996,2.577825,...,1,240.00,224.740005,224.375595,6.790066,6.963505,1,1,2. With Amazon (AMZN) up after its Prime Day p...,"Buy now, as the stock is likely to continue it..."
2,05-07-2024 14:30,TSLA,315.184998,0,0,1,310.23,295.929993,293.920013,4.832226,...,1,306.89,306.910004,310.979187,0.006518,1.314939,0,0,3. Tesla (TSLA) just announced another price c...,Hold
3,05-07-2024 14:30,AMD,137.949997,1,0,0,145.00,137.175003,134.830002,5.704390,...,1,155.00,144.070007,144.047302,7.586584,7.603542,1,1,4. After the recent pullback in Advanced Micro...,"Buy, as the recent pullback in AMD presents an..."
4,05-07-2024 14:30,META,718.599976,0,0,1,732.00,726.505005,718.640015,0.756360,...,1,760.00,724.315002,716.619995,4.926724,6.053418,1,1,5. Is Meta Platforms (META) still a good buy g...,"{'stock': 'META', 'reason': 'Meta Platforms, I..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,06-07-2024 14:30,ROST,131.649994,0,0,1,95.21,129.899994,130.770004,26.705154,...,0,98.01,129.750000,129.860001,24.462428,24.526413,0,0,95. What’s the ideal weight for a 4-stock mix ...,"{'ideal_weight': {'ASML': 30, 'KLAC': 25, 'INT..."
186,06-07-2024 14:30,EBAY,76.389999,0,1,0,74.50,75.288696,75.889999,1.047563,...,0,72.50,76.339996,76.339996,5.030124,5.030124,0,0,"99. I’ve held eBay for a while, but I’m consid...",{'coming_day': 'Hold your position in EBAY for...
187,06-07-2024 14:30,ADP,308.959991,0,0,0,313.00,310.725006,308.489990,0.732157,...,1,320.00,305.070007,304.149994,4.893956,5.211247,1,1,"100. I'm looking at ADP, NTES, and KLAC for lo...","Based on the analysis, I recommend investing i..."
188,06-07-2024 14:30,NTES,132.860001,1,0,0,137.00,134.154999,133.039993,2.120682,...,1,144.00,128.860001,128.259995,11.749185,12.271952,1,1,"100. I'm looking at ADP, NTES, and KLAC for lo...","Based on the analysis, I recommend investing i..."


In [37]:
df1 = yf.download("KO", interval = '1d', period='5y')
df2 = yf.download("PEP", interval = '1d', period='5y')

df1.columns = df1.columns.droplevel(1)
df2.columns = df2.columns.droplevel(1)
df1 = df1.reset_index()[['Date', 'Close', 'High', 'Low', 'Open', 'Volume']]
df2 = df2.reset_index()[['Date', 'Close', 'High', 'Low', 'Open', 'Volume']]


C:\Users\Azeem\AppData\Local\Temp\ipykernel_8364\2560039589.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df1 = yf.download("KO", interval = '1d', period='5y')
[*********************100%***********************]  1 of 1 completed
C:\Users\Azeem\AppData\Local\Temp\ipykernel_8364\2560039589.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df2 = yf.download("PEP", interval = '1d', period='5y')
[*********************100%***********************]  1 of 1 completed
